# Random Forest 2

En este segundo notebook hacemos random forest con Cross Validation para practicar la implementación.

In [ ]:
# paquetes
import pandas as pd # para maenejar dataframes
from sklearn.preprocessing import LabelEncoder # para codificar las categorías
from sklearn.model_selection import train_test_split # para dividir la base de datos en train/test
from sklearn.ensemble import RandomForestClassifier # para implementar random forest
from sklearn.model_selection import cross_validate # para hacer cross validation
from sklearn.model_selection import KFold # para la estrategia de división de cross validation
import matplotlib.pyplot as plt # para graficar
from numpy import mean # para sacar promedios

In [ ]:
# Repetimos los pasos del Random_Forest_1

# importamos los datos
df_import = pd.read_csv('df_datos.csv', index_col='Unnamed: 0')

# separamos las variables predictivas (X) de la categoría (y)
df_X = df_import.drop(columns = ['id_ecg','categoria','patient_id'])
ps_y = df_import['categoria'] # ps = pandas series

# label encoder
label_encoder = LabelEncoder()
ps_y_encoded = label_encoder.fit_transform(ps_y)

# vamos a dividir los datos para el grupo de entrenamiento y el de prueba.
# este punto es tricky. Vamos a hacer la cross validatión solo con los conjuntos de entrenamiento, el conjunto de test se deja aparte.
# no es tan necesario si no se va a hacer gridsearch
X_train, X_test, y_train, y_test = train_test_split(df_X, # variables predictivas
                                                    ps_y_encoded, # variable objetivo (ya codificada)
                                                    test_size= 0.8, # proporción del test size
                                                    random_state=10) # fijamos random seed para reproducibilidad

# creamos el objeto del modelo random forest
rf_abstracto = RandomForestClassifier(n_estimators=20, # número de árboles que se van a generar
                                      criterion='gini', # criterio de decisión para la construcción de cada árbol
                                      max_depth=None, # máximo número de nodos hacia abajo que puede haber. None = infinito
                                      min_samples_split=10, # si más de n eventos llegan a una hoja, entonces se vuelve en nodo y se subdivide en dos hojas.
                                      max_features=6, # cuántas columnas se van a considerar en la construcción de cada árbol.
                                      bootstrap=True, # usar subconjuntos con repetición de la base de datos para construír cada árbol.
                                      random_state=10) # semilla aleatoria para reproducibilidad

In [ ]:
# Cross Validation
# Cross validation va a dividir nuestros datos en n grupos iguales
# Despues va a entrenar n modelos usando todas las combinaciones de n-1 conjuntos
# Y evaluará cada modelo con el conjunto n-esimo que no usó para entrenar

# KFold es un método para definir la forma en la que queremos que queremos que cross validation divida los subconjuntos
cross_validation_strategy = KFold(n_splits=10, # número de subconjuntos o 'folds'
                                  shuffle=True, # si queremos que se shufleen los datos antes de dividirlos (es bueno ya que vienen ordenados)
                                  random_state=10) # semilla aleatoria

# ejecutamos cross validation y nos devuelve el score de cada modelo entrenado
cross_validation_scores = cross_validate(estimator = rf_abstracto, # el modelo con el cual entrenaremos, ya con hiperparámetros
                                         X = X_train, # usamos los conjuntos de entrenamiento
                                         y = y_train, # recordemos que ya tienen las y codificadas
                                         scoring = 'accuracy',
                                         cv = cross_validation_strategy, # aquí podemos dar un número de folds o un objeto que indique la estrategia de foldeo
                                         error_score='raise') # raise indica que en caso de error, nos avise y pare la ejecución

# enlistamos los resultados
lista_resultados = list(cross_validation_scores.get("test_score"))

#graficamos
plt.figure(figsize=(8, 6))
plt.bar([x for x in range(10)],lista_resultados, label = "precisión")
plt.axhline(mean(lista_resultados), c = 'red')
plt.title("Resultados cross validation")
plt.show()